<h3>04. nn.Modul로 구현하는 선형회귀</h3>
이전 챕터까지는 선형 회귀를 좀 더 직접적으로 이해하기 위해 가설, 비용 함수를 직접 정의해서 선형 회귀 모델을 구현했습니다. 이번에는 파이토치에서 이미 구현되어져 제공되고 있는 함수들을 불러오는 것으로 더 쉽게 선형 회귀 모델을 구현해보겠습니다.

예를 들어 파이토치에서는 선형 회귀 모델이 nn.Linear()라는 함수로, 또 평균 제곱오차가 nn.functional.mse_loss()라는 함수로 구현되어져 있습니다. 

<h>

<h4>1.단순 선형 회귀 구현하기</h4>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [3]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [6]:
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
#nn.Linear()는 입력의 차원, 출력의 차원을 인수로 받습니다.

model = nn.Linear(1,1)

위 torch.nn.Linear 인자로 1, 1을 사용하였습니다. 하나의 입력 에 대해서 하나의 출력 을 가지므로, 입력 차원과 출력 차원 모두 1을 인수로 사용하였습니다. model에는 가중치 W와 편향 b가 저장되어져 있습니다. 이 값은 model.parameters()라는 함수를 사용하여 불러올 수 있는데, 한 번 출력해보겠습니다.

In [5]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


2개의 값이 출력되는데 첫번째 값이 W고, 두번째 값이 b에 해당됩니다. 두 값 모두 현재는 랜덤 초기화가 되어져 있습니다. 그리고 두 값 모두 학습의 대상이므로 requires_grad=True가 되어져 있는 것을 볼 수 있습니다.

이제 옵티마이저를 정의합니다. model.parameters()를 사용하여 W와 b를 전달합니다.
학습률(learning rate)은 0.01로 정합니다.

In [7]:
#optimzier 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)


In [8]:
#전체 훈련 데이터에 대해 경사 하강법을 2000회 반복
nb_epochs= 2000
for epoch in range(nb_epochs+ 1):

    prediction = model(x_train)

    #cost계산
    cost = F.mse_loss(prediction,y_train) #파이토치에서 제공하는 평균 제곱 오차 함수

    #cost로 H(x)를 개선하는 부분
    #gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient계산
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))


Epoch    0/2000 Cost: 18.562185
Epoch  100/2000 Cost: 0.128051
Epoch  200/2000 Cost: 0.079128
Epoch  300/2000 Cost: 0.048896
Epoch  400/2000 Cost: 0.030215
Epoch  500/2000 Cost: 0.018671
Epoch  600/2000 Cost: 0.011538
Epoch  700/2000 Cost: 0.007129
Epoch  800/2000 Cost: 0.004406
Epoch  900/2000 Cost: 0.002722
Epoch 1000/2000 Cost: 0.001682
Epoch 1100/2000 Cost: 0.001040
Epoch 1200/2000 Cost: 0.000642
Epoch 1300/2000 Cost: 0.000397
Epoch 1400/2000 Cost: 0.000245
Epoch 1500/2000 Cost: 0.000152
Epoch 1600/2000 Cost: 0.000094
Epoch 1700/2000 Cost: 0.000058
Epoch 1800/2000 Cost: 0.000036
Epoch 1900/2000 Cost: 0.000022
Epoch 2000/2000 Cost: 0.000014


In [10]:
# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) 
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 

훈련 후 입력이 4일 때의 예측값 : tensor([[7.9926]], grad_fn=<AddmmBackward0>)


In [9]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9957]], requires_grad=True), Parameter containing:
tensor([0.0097], requires_grad=True)]


<h4>2. 다중 선형 회귀 구현하기</h4>
이제 nn.Linear()와 nn.functional.mse_loss()로 다중 선형 회귀를 구현해봅시다. 사실 코드 자체는 달라지는 건 거의 없는데, nn.Linear()의 인자값과 학습률(learning rate)만 조절해주었습니다.

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [13]:
# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1.
model = nn.Linear(3,1)

In [14]:
print(list(model.parameters()))


[Parameter containing:
tensor([[-0.5435,  0.3462, -0.1188]], requires_grad=True), Parameter containing:
tensor([0.2937], requires_grad=True)]


첫번째 출력되는 것이 3개의 w고, 두번째 출력되는 것이 b에 해당됩니다. 두 값 모두 현재는 랜덤 초기화가 되어져 있습니다. 그리고 두 출력 결과 모두 학습의 대상이므로 requires_grad=True가 되어져 있는 것을 볼 수 있습니다

In [15]:
#옵티마이저
optimizer = torch.optim.SGD(model.parameters(),lr=1e-5)


In [21]:
nb_epochs = 2000

for epoch in range(nb_epochs):

    prediction = model(x_train)

    cost = F.mse_loss(prediction,y_train)

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()
    
    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 0.354469
Epoch  100/2000 Cost: 0.351031
Epoch  200/2000 Cost: 0.347742
Epoch  300/2000 Cost: 0.344590
Epoch  400/2000 Cost: 0.341568
Epoch  500/2000 Cost: 0.338680
Epoch  600/2000 Cost: 0.335909
Epoch  700/2000 Cost: 0.333248
Epoch  800/2000 Cost: 0.330698
Epoch  900/2000 Cost: 0.328249
Epoch 1000/2000 Cost: 0.325899
Epoch 1100/2000 Cost: 0.323636
Epoch 1200/2000 Cost: 0.321464
Epoch 1300/2000 Cost: 0.319371
Epoch 1400/2000 Cost: 0.317362
Epoch 1500/2000 Cost: 0.315419
Epoch 1600/2000 Cost: 0.313553
Epoch 1700/2000 Cost: 0.311756
Epoch 1800/2000 Cost: 0.310018
Epoch 1900/2000 Cost: 0.308345


In [22]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 


훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.4109]], grad_fn=<AddmmBackward0>)


In [23]:
print(list(model.parameters()))



[Parameter containing:
tensor([[0.8376, 0.4564, 0.7125]], requires_grad=True), Parameter containing:
tensor([0.3207], requires_grad=True)]
